In [1]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

## 1.1 Постановка задачи
1) Решить две системы (одну с “точной” правой частью, вторую с изменённой)
2) Найти число обусловленности исходной матрицы
3) Посчитать фактическую погрешность и оценку погрешности

### Вариант 9

In [1]:
let a = matrix [
    [-401.64; 200.12]
    [1200.72; -601.76]
]

let bAccurate = vector [200.; -600.]
let bAltered = vector [199.; -601.]

### Решение

In [1]:
let solutionAccurate = a.Solve bAccurate
let solutionAltered = a.Solve bAltered

let condA = a.ConditionNumber()

let factError = (solutionAltered - solutionAccurate).L2Norm() / solutionAccurate.L2Norm()
let estimateError = ()

condA

## 1.2 Постановка задачи
1) Реализовать метод Гаусса с выбором главного элемента для решения СЛАУ
2) Реализовать метод Жордана для нахождения обратной матрицы
3) Реализовать метод LU-разложения для нахождения определителя матрицы

### Вариант 9

In [1]:
let a = matrix [
    [12.78572; 1.534675; -3.947418]
    [1.534675; 9.709232; 0.918435]
    [-3.947418; 0.918435; 7.703946]
]

let b = vector [9.60565; 7.30777; 4.21575]

### Метод Гаусса в выбором главных компонент

In [1]:
let solveGauss (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let n = matrix.RowCount
    let extendedMatrix = matrix |> Matrix.appendCol freeVector
    for k = 0 to n - 1 do
        let temp = extendedMatrix.[k, k]
        if abs temp > 1e-16 then
            for j = k + 1 to n do
                extendedMatrix.[k, j] <- extendedMatrix.[k, j] / temp
        for i = k + 1 to n - 1 do
            let temp = extendedMatrix.[i, k]
            for j = k to n do
                extendedMatrix.[i, j] <- extendedMatrix.[i, j] - extendedMatrix.[k, j] * temp 

    let solution = DenseVector.zero<float> n
    for i = n - 1 downto 0 do
        let mutable sum = 0.
        for j = i + 1 to n - 1 do
            sum <- sum + extendedMatrix.[i, j] * solution.[j]
        solution.[i] <- extendedMatrix.[i, n] - sum

    solution   

In [1]:
display <| solveGauss a b
display <| a.Solve b

### Метод Жордана для нахождения обратной матрицы

In [1]:
let inverseJordan (matrix: Matrix<float>) = 
    let n = matrix.RowCount
    let extendedMatrix = matrix.Append <| DenseMatrix.identity n
    for k = 0 to n - 1 do
        let temp = extendedMatrix.[k, k]
        if abs temp > 1e-16 then
            for j = k + 1 to 2 * n - 1 do
                extendedMatrix.[k, j] <- extendedMatrix.[k, j] / temp
        for i in [0 .. k - 1] @ [k + 1 .. n - 1] do
            let temp = extendedMatrix.[i, k]
            for j = k to 2 * n - 1 do
                extendedMatrix.[i, j] <- extendedMatrix.[i, j] - extendedMatrix.[k, j] * temp   
    
    extendedMatrix

In [1]:
display <| inverseJordan a
display <| a.Inverse()

### Метод LU-разложения для нахождения определителя матрицы

In [1]:
let detLU (matrix: Matrix<float>) = 
    let n = matrix.RowCount
    let l = DenseMatrix.zero<float> n n
    let u = DenseMatrix.identity<float> n

    for i = 0 to n - 1 do
        for j = i to n - 1 do
            let mutable sum = 0.
            for k = 0 to i - 1 do
                sum <- l.[j, k] * u.[k, i]
            l.[j, i] <- matrix.[j, i] - sum

        for j = i to n - 1 do
            let mutable sum = 0.
            for k = 0 to i - 1 do
                sum <- l.[i, k] * u.[k, j]
            u.[i, j] <- (matrix.[i, j] - sum) / l.[i, i]

    l.Diagonal() |> Seq.reduce (*)


In [1]:
display <| detLU a
display <| a.Determinant()